In [1]:
import pandas as pd
import numpy as np

In [19]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import MiniBatchKMeans

In [20]:
def reduce_img_colours(img_path, compression = 1, num_colours = 5):
    
    img = Image.open(img_path)
    
    new_size = (np.array(img.size) * compression).astype(int)
    
    img_array = np.array(img.resize(new_size))
    
    pixels = img_array.reshape((-1, 3))
    
    kmeans = MiniBatchKMeans(num_colours).fit(pixels)
    
    new_pixels = kmeans.cluster_centers_[kmeans.labels_]
    
    new_img = new_pixels.reshape((new_size[1], new_size[0], 3))
    
    print(f'# Old colours: {len(np.unique(pixels, axis = 0))}')
    print(f'# New colours: {num_colours}')
    
    return new_img

In [26]:
# https://github.com/google-research-datasets/conceptual-captions/blob/master/LICENSE
# https://ai.google.com/research/ConceptualCaptions/download
data = pd.read_csv('../raw_data/Validation_GCC-1.1.0-Validation.tsv', sep = '\t', header = None)

In [27]:
data.columns = ['caption', 'url']

In [145]:
import requests
from PIL import Image
import io
import torch
import clip

def get_pil_image(url):
    
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return Image.open(io.BytesIO(response.content))
        
    except requests.exceptions.RequestException:
        return None


def square_image(pil_img):
    
    short_side = min(pil_img.size)
    
    img_width, img_height = pil_img.size
    return pil_img.crop(((img_width - short_side) // 2,
                         (img_height - short_side) // 2,
                         (img_width + short_side) // 2,
                         (img_height + short_side) // 2))

def simplify_image(pil_img, num_colors = 8):

    img = pil_img.quantize(colors=num_colors, method=2)
    
    sq_img = square_image(img).resize((256, 256), Image.LANCZOS)
    
    return sq_img
    
def save_img(pil_img, caption, destination_folder):
    
    file_name = f"{destination_folder}/{caption.replace('.', '')}.png"
    
    print('saving to ', file_name)
    
    pil_img.save(file_name)


model, preprocess = clip.load("ViT-B/32", device=device)


def clip_encode_image(pil_img):
    
    device = "cuda" if torch.cuda.is_available() else "cpu"
    
    image = preprocess(pil_img).unsqueeze(0).to(device)

    with torch.no_grad():
        image_features = model.encode_image(image)

    return image_features



In [146]:
features_list = []
image_path_list = []

for i, row in data[5:10].iterrows():
    
    url, caption = row['url'], row['caption']
    
    print(url)

    img = get_pil_image(url)
    
    if img:

        features_list.append(clip_encode_image(img)[0])

        simple_img = simplify_image(img, num_colors = 256)

        save_img(simple_img, caption, destination_folder = '../raw_data/images')
        
        image_path_list.append(f"{caption.replace('.', '')}.png")

#image = get_simplified_image(url, caption, destination_folder = '../raw_data/images', num_colors = 256)

http://i.dailymail.co.uk/i/pix/2014/11/05/1415187324676_wps_31_Home_is_a_little_Deer_Ivy.jpg
saving to  ../raw_data/images/the - bedroom stone cottage can sleep people.png
http://www.waste360.com/sites/waste360.com/files/styles/article_featured_standard/public/Trista%2002%20007_0.jpg?itok=F1eJZsX3
https://media.gettyimages.com/photos/young-woman-seated-on-the-beach-picture-id97545987?s=612x612
https://worldjourneysdiscover.files.wordpress.com/2014/07/kyoto-07.jpg?w=860&h=645
saving to  ../raw_data/images/party in the park under cherry blossoms.png
http://piquemagazine.uk/wp-content/uploads/2017/10/LPO-24-Feb-Albrecht-Menzel-%C2%AE-Anne-Hornemann-300dpi.jpg


In [149]:
image_embeddings = np.stack(features_list)

In [150]:
image_embeddings

array([[-0.23239458,  0.06971791,  0.09149291, ...,  0.31141508,
         0.05654181, -0.37589613],
       [ 0.03647149,  0.16184269, -0.20005846, ...,  0.04125442,
        -0.22502679,  0.37220034]], dtype=float32)

In [151]:
!ls '../raw_data/images'

author : a life in photography -- in pictures.png
party in the park under cherry blossoms.png
the - bedroom stone cottage can sleep people.png
tv police procedural is filming on the street this week .png
tv police procedural is filming on the street this week _256bit.png
tv police procedural is filming on the street this week _2bit.png
tv police procedural is filming on the street this week _8bit.png
